In [ ]:
#TODO: write explanation, what's going on here
  # to be more precise:
  #  we are training the model for the search
  #  our embeddings
  #  our datasets
  #  our model that we would like to train
  #  it will be great if the code could be able to be executed on colab - think about it

In [16]:
! wget -O quora_questions.zip https://dl.fbaipublicfiles.com/glue/data/QQP-clean.zip && \
  tar xzvf quora_questions.zip && \
  rm quora_questions.zip && \
  mkdir -p resources && \
  mv QQP/ quora_questions

--2022-09-10 14:49:58--  https://dl.fbaipublicfiles.com/glue/data/QQP-clean.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41696084 (40M) [application/zip]
Saving to: ‘resources/quora_questions.zip’

resources/quora_que 100%[===================>]  39.76M  2.84MB/s    in 15s     

2022-09-10 14:50:13 (2.73 MB/s) - ‘resources/quora_questions.zip’ saved [41696084/41696084]

x QQP/
x QQP/train.tsv
x QQP/dev.tsv
x QQP/test.tsv


In [9]:
! rm -rf resources

In [5]:
! tar xzvf QQP-clean.zip

x QQP/
x QQP/train.tsv
x QQP/dev.tsv
x QQP/test.tsv


In [6]:
! ls QQP

dev.tsv   test.tsv  train.tsv


In [23]:
! wget http://nlp.stanford.edu/data/glove.6B.zip

--2022-09-10 14:56:53--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-09-10 14:56:53--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-09-10 14:56:54--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

gl

In [ ]:
! 

In [ ]:
! tar xzvf glove.6B.zip -C 

In [ ]:
# $ tar -zxvf config.tar.gz etc/default/sysstat
# $ tar -xvf {tarball.tar} {path/to/file}
# glove.6B.50d.txt

! tar http://nlp.stanford.edu/data/glove.6B.zip

tar -zxvf config.tar.gz etc/default/sysstat

In [ ]:
# download the